This notebook takes in bike counts and bike counter csvs and performs the following:
* trims the bike counters to the predominant counters and removes duplicative counters
* links together the two Kent Ave counters
* slices off anomalous data that occurs after counter installation
* groups by by hour and week for visualization
* creates display data for streamlit app

In [5]:
# data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# geo
import geopandas as gpd
from shapely.geometry import Point

# utils
import pickle
from datetime import datetime

import_new_data=True

## clean data

In [6]:
if import_new_data==True:
    import_df = pd.read_csv('data/Bicycle_Counts.csv')
    import_df['date'] = pd.to_datetime(import_df['date'])
    import_df = import_df.set_index('date')
    
    date_retrieved = datetime.now() # record datetime of import
    with open('data/retrieval_date.pkl', 'wb') as f:
        pickle.dump(date_retrieved, f)

#status col:
#0 = raw
#1 = excluded 
#2 = deleted
#4 = modified
#8 = validated
#16 = certified

/var/folders/4p/cnm8c74x2cscn3bny0pr_bvr0000gn/T/ipykernel_35787/1960095748.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  import_df = pd.read_csv('data/Bicycle_Counts.csv')


In [7]:
# copy after imported to save having to import again
data = import_df[['id','counts']].copy()

# the raw file has comma thousands separators pandas is unable to interpret as ints (in this instance)
# remove commas and cast as int
data.loc[data['counts'].str.contains(',')==True, 'counts'] = data.loc[data['counts'].str.contains(',')==True, 'counts'].str.replace(',','')
data['counts'] = pd.to_numeric(data['counts']).astype(int)
data

,id,counts
date,,
2018-12-24 03:30:00,100009424,0
2018-12-24 03:45:00,100009424,0
2018-12-24 04:00:00,100009424,0
2018-12-24 04:15:00,100009424,0
2018-12-24 04:30:00,100009424,0
...,...,...
2023-09-12 22:45:00,300029648,18
2023-09-12 23:00:00,300029648,18
2023-09-12 23:15:00,300029648,15


In [9]:
# read in counters
counters = pd.read_csv('data/Bicycle_Counters.csv')
counters = counters[['id', 'name', 'latitude', 'longitude']]

# merge counter names
data = data.reset_index().merge(counters[['id', 'name']], on='id').set_index('date')

In [10]:
# select the counters to ultimately analyze

# drop counters that are tests
test_counters = [300020692, 100048744, 100005020, 100055175]
# Drop pedestrian counters
ped_counters = [300029648, 100009426]
# Drop other counters
# bk bridge comprehensive is all we need for the bridge
other_counters = [300020241, 100010022, 300028963, 100051865, 100009429, 100039064, 100057318, 100047029]

counters_to_remove = test_counters + ped_counters + other_counters

data = data[~data['id'].isin(counters_to_remove)]
counters = counters[~counters['id'].isin(counters_to_remove)]


#rename n.8th kent because we replace it with kent ave comprehensive below
counters.loc[counters['name']=='Kent Ave btw North 8th St and North 9th St', ['name', 'id',]] = ('Kent Ave Comprehensive', 999999999)
counters = counters[counters['name']!='Kent Ave btw South 6th St. and Broadway']

In [11]:
# check overlap of counters
# get the first and last date of each counter
first_dates = data.reset_index().groupby('name')['date'].min()
last_dates = data.reset_index().groupby('name')['date'].max()

counter_dates = pd.DataFrame({'first': first_dates, 'last':last_dates}).sort_values(by='last')
counter_dates['runtime'] = (counter_dates['last'] - counter_dates['first']).dt.days
counter_dates.sort_values(by='runtime')

,first,last,runtime
name,,,
Willis Ave Bikes,2022-09-02 11:15:00,2023-09-12 23:45:00,375
Fountain Ave,2022-08-23 10:30:00,2023-09-12 23:45:00,385
111th St at 50th Ave,2022-05-09 15:00:00,2023-09-11 23:45:00,490
Kent Ave btw South 6th St. and Broadway,2014-08-14 00:00:00,2016-11-21 23:45:00,830
Amsterdam Ave at 86th St.,2019-10-24 00:00:00,2023-09-10 23:45:00,1417
Columbus Ave at 86th St.,2019-10-24 00:00:00,2023-09-11 23:45:00,1418
2nd Avenue - 26th St S,2015-05-22 09:15:00,2019-06-11 23:45:00,1481
8th Ave at 50th St.,2018-06-14 00:00:00,2023-09-11 23:45:00,1915
Pulaski Bridge,2017-06-24 00:00:00,2023-09-12 23:45:00,2271


In [12]:
# the two kent ave counters about 10 blocks from one another and they ended/started one day apart, let's combine them
data.loc[data['name']=='Kent Ave btw South 6th St. and Broadway', ['name', 'id']] = ('Kent Ave Comprehensive', 999999999)
data.loc[data['name']=='Kent Ave btw North 8th St and North 9th St', ['name', 'id']] = ('Kent Ave Comprehensive', 999999999)

### clean anomalous data

In [13]:
# several counters have odd spikes, remove them 
# 8th ave and 50th st. reads zero for the first year and then spikes up to 22k in a single day throughout July
# could not find events that occurred in that July to cause a spike
# deleting everything before 7/24/2019
eighth_criteria = (data['id']==100057316) & (data.index < '2019-07-24')
data = data[~eighth_criteria].copy()

amsterdam_criteria = (data['id']==100057319) & (data.index < '2020-12-15')
data = data[~amsterdam_criteria].copy()

first_ave_criteria = (data['id']==100010020) & ((data.index<'2016-08-10') | (data.index>'2019-06-11'))
data = data[~first_ave_criteria].copy()

second_ave_criteria = (data['id']==100009424) & (data.index > '2018-07-14')
data = data[~second_ave_criteria].copy()

data = data.sort_index()

### replace with pretty names

In [19]:
# pretty names for brevity and viewing
pretty_names = {
    '2nd Avenue - 26th St S': '26th St & 2nd Ave', 
    'Prospect Park West':'Prospect Pk W',
    'Williamsburg Bridge Bike Path': 'Williamsburg Br',
    'Ed Koch Queensboro Bridge Shared Path':'Queensboro Br', 
    'Staten Island Ferry':'Staten Isl Ferry St',
    'Pulaski Bridge':'Pulaski Br', 
    '1st Avenue - 26th St N - Interference testing': '1st Ave & 26th St',
    '8th Ave at 50th St.': '8th Ave & 50th St',
    'Amsterdam Ave at 86th St.':'Amsterdam Ave & 86th St', 
    'Manhattan Bridge Bike Comprehensive':'Manhattan Br',
    'Comprehensive Brooklyn Bridge Counter':'Brooklyn Br', 
    'Fountain Ave':'Fountain Ave',
    'Columbus Ave at 86th St.':'Columbus Ave & 86th St',
    'Kent Ave Comprehensive': 'Kent Ave', 
    '111th St at 50th Ave':'111th St & 50th Ave',
    'Willis Ave Bikes':'Willis Ave'
}

counters['name'] = counters['name'].map(pretty_names)
data['name'] = data['name'].map(pretty_names)

In [20]:
# bike prediction app will use these counters
# export a dict of their names and ids for display purposes
coun = ['Manhattan Br', 'Brooklyn Br', 'Williamsburg Br', 'Kent Ave', 'Queensboro Br', 'Prospect Park W']
counter_dict = {}
for c in coun:
    if c == 'Prospect Park W':
        counter_dict[c] = 100009425
    else:
        counter_dict[c] = float(counters.loc[counters['name']==c, 'id'].values[0])

In [21]:
# save
counters.set_index('id').to_pickle('data/counters.pkl')

with open('bike-prediction-app/data/counter_dict.pkl', 'wb') as f:
    pickle.dump(counter_dict, f)

# groupby hour

In [453]:
# first aggregate up the 15 min counts to the hour level
# group by week but preserve the year so we have all historical data
historical_hr = data.reset_index()[['date', 'id', 'counts', 'name']]\
                    .groupby([pd.Grouper(key='date', freq='H'), 'id', 'name'])\
                    .sum()\
                    .reset_index()

In [455]:
# get hour of day for display
def get_time(hour):
    hour_string = str(hour).zfill(2)
    time_string = hour_string + ':00' + ':00'
    return pd.to_datetime(time_string, format='%H:%M:%S', utc=True)

In [456]:
#groupby hour and take mean
hr = historical_hr.groupby([historical_hr['date'].dt.hour, 'id', 'name']).mean()
hr['display_time'] = hr.index.get_level_values('date').to_series().apply(lambda x: get_time(x)).values

In [457]:
# save
hr.to_pickle('data/by_hour.pkl')

## groupby week

In [458]:
# group by week but preserve the year so we have all historical data
historical_wk = data.reset_index()[['date', 'id', 'counts', 'name']]\
                    .groupby([pd.Grouper(key='date', freq='W-MON'), 'id', 'name'])\
                    .sum()

In [459]:
# get month of year given week number
def get_month(week_number, year):
    first_day = datetime.fromisocalendar(year, week_number, 1)
    return first_day.strftime('%m-%d')

In [460]:
historical_wk_noidx = historical_wk.reset_index() # remove index for groupby
wk = historical_wk_noidx.groupby([historical_wk_noidx['date'].dt.week,'id', 'name']).mean() # group by week
wk = wk.loc[wk.index.get_level_values('date')!=53] # get rid of 53rd week
# create display dates for charts
wk['display_date'] = wk.index.get_level_values('date').to_series().apply(lambda x: get_month(x,2021)).values 

/var/folders/4p/cnm8c74x2cscn3bny0pr_bvr0000gn/T/ipykernel_82121/809708603.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  wk = historical_wk_noidx.groupby([historical_wk_noidx['date'].dt.week,'id', 'name']).mean()


In [461]:
# save
wk.to_pickle('data/by_week.pkl')

## make app data

In [462]:
# to save time on the color mapping, map colors ahead of time

# colors
cat_20 = ['#1f77b4',
    '#aec7e8',
    '#ff7f0e',
    '#ffbb78',
    '#2ca02c',
    '#98df8a',
    '#d62728',
    '#ff9896',
    '#9467bd',
    '#c5b0d5',
    '#8c564b',
    '#c49c94',
    '#e377c2',
    '#f7b6d2',
    '#7f7f7f',
    '#c7c7c7',
    '#bcbd22',
    '#dbdb8d',
    '#17becf',
    '#9edae5'
    ]

num_counters = len(counters)
counter_ids = counters['id'].unique()
color_indices = np.linspace(0, len(cat_20)-1, num_counters, dtype=int)
colors = [cat_20[x] for x in color_indices]
color_dict = dict(zip(counter_ids, colors))

In [463]:
# map colors and save
counter_display = counters.copy()
counter_display['color'] = counter_display['id'].map(color_dict)
counter_display.set_index('id').to_pickle('bike-counts-app/data/streamlit_counters.pkl')

hr_display = hr.copy()
hr_display['color'] = hr_display.index.get_level_values('id').map(color_dict)
hr_display.to_pickle('bike-counts-app/data/streamlit_by_hr.pkl')

wk_display = wk.copy()
wk_display['color'] = wk_display.index.get_level_values('id').map(color_dict)
wk_display.to_pickle('bike-counts-app/data/streamlit_by_wk.pkl')

historical_wk_display = historical_wk.copy()
historical_wk_display['id'] = historical_wk_display.index.get_level_values('id').astype(int)
historical_wk_display['color'] = historical_wk_display['id'].map(color_dict)
historical_wk_display = historical_wk_display.drop(columns=['id'])
cutoff = '2015-01-01'
historical_wk_display = historical_wk_display.loc[historical_wk_display.index.get_level_values('date') >= cutoff ]
historical_wk_display.to_pickle('bike-counts-app/data/streamlit_hist_by_wk.pkl')

data.to_pickle('bike-counts-app/data/all_data.pkl')